In [64]:
import keras
from keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt
from keras.models import load_model

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.optimizers import SGD

In [2]:
mnist_data = mnist.load_data()

In [3]:
def make_noisy(mnist_image):
    mnist_rand_image = np.copy(mnist_image)

    for x in np.nditer(mnist_rand_image, op_flags=['readwrite']):
        randval = np.random.randint(0,10)
        if randval == 0:
            x[...] = 0
        elif randval == 1:
            x[...] = 255
            
    return mnist_rand_image

#plt.imshow(make_noisy(mnist_data[0][0][0]), cmap='gray')

In [4]:
train_noisy_clean_array = []
train_data_size = mnist_data[0][0].shape[0]
validation_split_size = 0.2
actual_train_data_size = int(train_data_size * (1 - validation_split_size))

# Training set
for i in range(0, actual_train_data_size):
    if (i < actual_train_data_size/2):
        train_noisy_clean_array.append(make_noisy(mnist_data[0][0][i]))
    else:
        train_noisy_clean_array.append(mnist_data[0][0][i])

# Validation set 
validation_data_size = train_data_size - actual_train_data_size

for i in range(0, validation_data_size):
    if (i < validation_data_size/2):
        train_noisy_clean_array.append(make_noisy(mnist_data[0][0][i+actual_train_data_size]))
    else:
        train_noisy_clean_array.append(mnist_data[0][0][i+actual_train_data_size])

train_noisy_clean_array = np.asarray(train_noisy_clean_array)

In [5]:
train_labels=mnist_data[0][1]
test_labels=mnist_data[1][1]

In [6]:
noisy_then_clean_mnist = ((train_noisy_clean_array,train_labels),(mnist_data[1][0],test_labels))

In [8]:
half_noisy_mnist_train = noisy_then_clean_mnist[0][0][0:24000]

In [9]:
half_clean_mnist_train = noisy_then_clean_mnist[0][0][24000:48000]

In [10]:
half_noisy_mnist_validation = noisy_then_clean_mnist[0][0][48000:48000+6000]

In [11]:
half_clean_mnist_validation = noisy_then_clean_mnist[0][0][48000+6000:60000]

In [32]:
noisy_mnist_half_train_validation = np.concatenate((half_noisy_mnist_train, half_noisy_mnist_validation), axis=0)

In [31]:
clean_mnist_half_train_validation = np.concatenate((half_clean_mnist_train, half_clean_mnist_validation), axis=0)

In [41]:
labels_for_noisy_train_validation = np.concatenate((train_labels[0:24000],train_labels[48000:48000+6000]))

In [42]:
labels_for_clean_train_validation = np.concatenate((train_labels[24000:48000],train_labels[48000+6000:60000]))

In [59]:
half_noisy_mnist = ((noisy_mnist_half_train_validation, labels_for_noisy_train_validation), (mnist_data[1][0], mnist_data[1][1]))

In [76]:
half_clean_mnist = ((clean_mnist_half_train_validation,labels_for_clean_train_validation), (mnist_data[1][0], mnist_data[1][1]))

In [81]:
## NOISY_THEN_CLEAN MNIST

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.optimizers import SGD

batch_size = 128
num_classes = 10
epochs = 20

## Clean portion

# the data, shuffled and split between train and test sets
print("Clean data")

(x_train, y_train), (x_test, y_test) = half_clean_mnist #mnist.load_data() 

x_train = x_train.reshape(30000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

sgd = SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Noisy portion

print("Noisy data")

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = half_noisy_mnist #mnist.load_data() 

x_train = x_train.reshape(30000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model.summary()

sgd = SGD(lr=0.03, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.2)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

model.save("keras_mnist_modelv2_one_dense_clean_then_noisy.h5")

Clean data
30000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Train on 24000 samples, validate on 6000 samples
Epoch 1/20
24000/24000 [==============================] - 1s 53us/step - loss: 0.4791 - acc: 0.8647 - val_loss: 0.2277 - val_acc: 0.9338
Epoch 2/20
24000/24000 [==============================] - 1s 46us/step - loss: 0.2390 - acc: 0.9305 - val_loss: 0.1719 - val_acc: 0.9535
Epoch 3/20
24000/24000 [==============================] - 1s 50us/step - loss: 0.1820 - acc: 0.9470 - val_loss: 0.1380 - val_acc: 0.9620
Epoch 4/20
24000/24000

In [83]:
train_noisy_array = []
train_data_size = mnist_data[0][0].shape[0]

for i in range(0,train_data_size):
    train_noisy_array.append(make_noisy(mnist_data[0][0][i]))

train_noisy_array = np.asarray(train_noisy_array)

In [84]:
test_noisy_array = []
test_data_size = mnist_data[1][0].shape[0]

for i in range(0,test_data_size):
    test_noisy_array.append(make_noisy(mnist_data[1][0][i]))

test_noisy_array = np.asarray(test_noisy_array)

In [85]:
train_labels=mnist_data[0][1]

In [86]:
test_labels=mnist_data[1][1]

In [87]:
noisy_mnist = ((train_noisy_array, train_labels),(test_noisy_array, test_labels))

In [92]:
model = load_model("keras_mnist_modelv2_one_dense_clean_then_noisy.h5")

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = noisy_mnist #mnist.load_data() 

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model.summary()

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

60000 train samples
10000 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_10 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.19710883660701803
Test accuracy: 0.9451


## On clean mnist, the noisy_then_clean neural network achieved an accuracy of 0.9708

## One noisy mnist, the noisy_then_clean neural network achieved an accuracy of 0.9451